# SDND Project 2 - Finding Lane Lines Advanced

In [1]:
# ------------------------------------------------- Import ------------------------------------------------- # 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import os
# ------------------------------------------------- Import ------------------------------------------------- # 

### Camera Calibration - Chessboard

In [2]:
# ------------------------------------------------- Camera Calibration ------------------------------------------------- # 
def Chessboard_Calibrate():
    
    objp = np.zeros((6*9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    
    objpoints = []
    imgpoints = []
    
    images = glob.glob('camera_cal/calibration*.jpg')
    
    for idx, fname in enumerate(images):
        
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
        
        if ret == True:
            
            objpoints.append(objp)
            imgpoints.append(corners)
            
            cv2.drawChessboardCorners(img, (9, 6), corners, ret)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)  
                
    return mtx, dist

# ------------------------------------------------- Camera Calibration ------------------------------------------------- # 

### Create Binary Threshold Image

In [3]:
# ------------------------------------------------- Binary Threshold Image ------------------------------------------------- # 
def Binary_Threshold(img):
    
    # R-channel & S-channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    r_channel = img[:, :, 0]
    s_channel = hls[:, :, 2]
    
    # Sobel binary x-gradient
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=11)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    thresh_min = 20
    thresh_max = 100
    
    sb_binary = np.zeros_like(scaled_sobelx)
    sb_binary[(scaled_sobelx >= thresh_min) & (scaled_sobelx <= thresh_max)] = 1
    
    # S-binary
    s_thresh_min = 170
    s_thresh_max = 255
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    # R-binary
    r_thresh_min = 225
    r_thresh_max = 255
    
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh_min) & (r_channel <= r_thresh_max)] = 1
    
    combined_binary = np.zeros_like(sb_binary)
    
    combined_binary[(r_binary == 1) | (sb_binary == 1) | (s_binary == 1)] = 1

    binary_output = combined_binary * 255
    
    return binary_output

# ------------------------------------------------- Binary Threshold Image ------------------------------------------------- # 

In [4]:
# BT_Test()

In [5]:
# # Test Threshold
# def BT_Test():
    
# #     cap = cv2.VideoCapture('challenge_video.mp4')
#     cap = cv2.VideoCapture('challenge_video.mp4')
# #     cap = cv2.VideoCapture('project_video.mp4')


#     if(cap.isOpened):
#         width  = int(cap.get(3))
#         height = int(cap.get(4)) 

#     img_size = (width, height)

#     while(cap.isOpened):

#         ret, frame = cap.read()        
#         if(ret == False):
#             break

#         binimg = Binary_Threshold(frame)

#         cv2.imshow('Result', binimg) # Test bin_thresh

#         if(cv2.waitKey(40) & 0xFF == ord('q')):
#             break

#     cap.release()
#     cv2.destroyAllWindows()
    
#     return 'Done!'

### Finding Lane Pixels and Fit Polynomial

In [6]:
# ------------------------------------------------- Find Lane Lines Utils ------------------------------------------------- # 
def Find_Lane_Pixels(img): 
    
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    nwindows = 9
    margin = 60
    minpix = 50
    
    window_height = np.int(img.shape[0]//nwindows)
    
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        
    except ValueError:
        
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

# ------------------------------------- #

def Fit_Polynomial(img):
    
    leftx, lefty, rightx, righty = Find_Lane_Pixels(img)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    
    try:
        
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        foundFlag = True
        
    except TypeError:
        
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        foundFlag = False

    return left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag

# ------------------------------------- # 

def Fit_Poly(img_shape, leftx, lefty, rightx, righty):
    
    # Fit a second order polynomial to each with np.polyfit()
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    # Calc both polynomials using ploty, left_fit and right_fit 
    try: 
        
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        foundFlag = True
        
    except TypeError: 
        
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        foundFlag = False
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag

# ------------------------------------- #

def Search_Around_Poly(binary_warped, left_fit, right_fit):

    # Width of the margin around the previous polynomial
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the area of search based on activated x-values 
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag = Fit_Poly(binary_warped.shape, leftx, lefty, rightx, righty) 

    return left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag

# ------------------------------------------------- Find Lane Lines Utils ------------------------------------------------- # 

### Pipeline - Videos (Main)

In [7]:
# ------------------------------------------------- Line Class ------------------------------------------------- # 
class Line():
    
    def __init__(self):
        
        # Was the line detected in the last iteration?                      
        self.detected = False  
        
        # x values of the last n fits of the line                         
        self.recent_xfitted = [] 
        
        # Average x values of the fitted line over the last n iterations          
        self.bestx = None     
        
        # Polynomial coefficients for the recent n fits                      
        self.recent_fits = []    
        
        # Polynomial coefficients averaged over the last n iterations             
        self.best_fit = None  
        
        # Radius of curvature of the line in some units                            
        self.radius_of_curvature = None 
        
        # Distance in meters of vehicle center from the line                       
        self.line_base_pos = None 
        
        # Difference in fit coefficients between last and new fits                 
        self.diffs = np.array([0,0,0], dtype='float')  
        
# ------------------------------------------------- Line Class ------------------------------------------------- # 

In [8]:
# ------------------------------------------------- Measure Curvature ------------------------------------------------- # 
def Calculate_Curvature(ploty, left_fit, right_fit):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30.0/720
    xm_per_pix = 3.7/700
    
    # Bottom of the image
    y_eval = np.max(ploty)
    
    if ((left_fit is not None) and (right_fit is not None)):
        # Calculate radius
        left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        # Calculate center_distance
        midx = 640

        left_bottom = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
        right_bottom = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]

        lane_center = (left_bottom + right_bottom) / 2
        center_dist = (midx - lane_center) * xm_per_pix
        
    else: 
        print('[2] Curvature calculation failed!')
        return 0
    
    return left_curverad, right_curverad, center_dist

# ------------------------------------------------- Measure Curvature ------------------------------------------------- # 

In [9]:
# ------------------------------------------------- Visualization ------------------------------------------------- # 
def Unwarp_Detected_Lane(warped, left_fitx, right_fitx, ploty, Minv):
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    
    return newwarp

# ------------------------------------------------- Visualization ------------------------------------------------- # 

In [10]:
# ------------------------------------------------- Lane Main ------------------------------------------------- # 
def Lane_Processings(lane, binary_warped, iter_n, Minv, undist): 

    lineL = lane[0]
    lineR = lane[1]
    
    # Sanity tolerance
    sanThresh = [0.001, 1.0, 100.0]
    
    # If no lane found in the last frame - Sliding window
    if ((lineL.detected == False) and (lineR.detected == False)):
        
        # Use sliding window to find initial fits
        left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag = Fit_Polynomial(binary_warped)

    # If lane detected in the last frame - Search around prior 
    else: 
        
        # Use search around prior to find new fits
        left_fitx, right_fitx, ploty, left_fit, right_fit, foundFlag = \
            Search_Around_Poly(binary_warped, lineL.recent_fits[-1], lineR.recent_fits[-1])
   
    # Update line.detected
    lineL.detected = foundFlag
    lineR.detected = foundFlag
    
    # If found fits
    if (foundFlag == True):
        
        # Check if new fit has similar shape
        sanFlag = True
        
        if ((lineL.best_fit is not None) and (lineR.best_fit is not None)):
            
            lineL.diffs = abs(left_fit - lineL.best_fit)
            lineR.diffs = abs(right_fit - lineR.best_fit)
            
            if (                                     \
                ((lineL.diffs[0] > sanThresh[0])  or \
                 (lineL.diffs[1] > sanThresh[1])  or \
                 (lineL.diffs[2] > sanThresh[2])) or \
                ((lineR.diffs[0] > sanThresh[0])  or \
                 (lineR.diffs[1] > sanThresh[1])  or \
                 (lineR.diffs[2] > sanThresh[2]))    \
               ):
                
                print('Sanity check failed, using the latest data!')
                
                sanFlag = False
                return lane            
                
        else: # The first frame
            
            lineL.recent_fits.append(left_fit)
            lineL.recent_xfitted.append(left_fitx)
            lineL.best_fit = left_fit
            lineL.bestx = left_fitx
            
            lineR.recent_fits.append(right_fit)          
            lineR.recent_xfitted.append(right_fitx)          
            lineR.best_fit = right_fit
            lineL.bestx = right_fitx
            
            print('ALF STARTED!')
            
        # -------------------- End of sanity check -------------------- #
        
        # Calculate curvature
        left_curverad, right_curverad, center_dist = Calculate_Curvature(ploty, left_fit, right_fit)
        
        lineL.radius_of_curvature = left_curverad
        lineL.line_base_pos = center_dist
        
        lineR.radius_of_curvature = right_curverad
        lineR.line_base_pos = center_dist
        
        # Update current fit               
        lineL.recent_fits.append(left_fit)
        lineR.recent_fits.append(right_fit)

        # Add new fit to list     
        lineL.recent_xfitted.append(left_fitx)
        lineR.recent_xfitted.append(right_fitx)

        # If number of fits > n
        if ((len(lineL.recent_fits) > iter_n) and (len(lineR.recent_fits) > iter_n)):

            # Pop out the oldest fit    
            lineL.recent_fits.pop(0)
            lineR.recent_fits.pop(0)
            
            # Pop out the oldest fitx
            lineL.recent_xfitted.pop(0)
            lineR.recent_xfitted.pop(0)

        # Update average
        lineL.best_fit = np.average(lineL.recent_fits, axis=0)
        lineR.best_fit = np.average(lineR.recent_fits, axis=0)
        
        lineL.bestx = np.average(lineL.recent_xfitted, axis=0)
        lineR.bestx = np.average(lineR.recent_xfitted, axis=0)

    # If no fit found - skip the frame
    else: 
        print('[1] Functions cannot find lines, using the latest data!')    
        return lane
    
    laneU = [lineL, lineR]

    return laneU

# ------------------------------------------------- Lane Main ------------------------------------------------- # 

In [11]:
# ------------------------------------------------- MAIN ------------------------------------------------- # 
def P2_Main(video, videoOut):
    
    # --- 1. Read in video. --- #
    cap = cv2.VideoCapture(video)
    
    # --- 2. Get frame size. --- # 
    if(cap.isOpened):
        width  = int(cap.get(3))
        height = int(cap.get(4)) 
    img_size = (width, height)
    
    ploty = np.linspace(0, 719, 720)
    
    # --- 3. Init video writer. --- #
    fourcc = 0x7634706d
    out = cv2.VideoWriter(videoOut, fourcc, 25.0, img_size)
    
    # --- 4. Persp. transform points. --- # 
    Psrc = np.float32([
        [(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
        [((img_size[0] / 6) + 5), img_size[1]],
        [(img_size[0] * 5 / 6) + 23, img_size[1]],
        [(img_size[0] / 2 + 58), img_size[1] / 2 + 100]
    ])
    Pdst = np.float32([
        [(img_size[0] / 4), 0],
        [(img_size[0] / 4), img_size[1]],
        [(img_size[0] * 3 / 4), img_size[1]],
        [(img_size[0] * 3 / 4), 0]
    ])
    
    # --- 5. Params & vars. --- # 
    Umtx, Udist = Chessboard_Calibrate()
    PM = cv2.getPerspectiveTransform(Psrc, Pdst)
    PMinv = cv2.getPerspectiveTransform(Pdst, Psrc)
    
    lineL = Line()
    lineR = Line()
    lane = [lineL, lineR]
    
    iter_n = 9
    
    # --- 6. Each frame in video. --- # 
    while(cap.isOpened):
        
        ret, frame = cap.read()        
        if(ret == False):
            break
        
        # -- 6.1. Binary threshold. -- #
        undistorted = cv2.undistort(frame, Umtx, Udist, None, Umtx)
        
        # -- 6.2. Undistort binimg. -- #
        
        binimg = Binary_Threshold(undistorted)
        
        # -- 6.3. Warp undistorted. -- #
        binary_warped = cv2.warpPerspective(binimg, PM, img_size, flags=cv2.INTER_LINEAR)
        
        # -- 6.4. Main processing: identify lane, measure curvature, sanity check. -- #
        laneU = Lane_Processings(lane, binary_warped, iter_n, PMinv, undistorted)
        lane = laneU        
        
        # -- 6.5. Visualization. -- #
        newwarp = Unwarp_Detected_Lane(binary_warped, lane[0].bestx, lane[1].bestx, ploty, PMinv)
        result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
        
        # Display curvature radius and distance between lane center
        font = cv2.FONT_HERSHEY_DUPLEX
        
        curv_rad = (lane[0].radius_of_curvature + lane[1].radius_of_curvature) / 2
        center_dist = lane[0].line_base_pos
        
        if (center_dist < 0):
            direction = 'LEFT'
        else:
            direction = 'RIGHT'
        
        text1 = 'Curve Radius: ' + '{:04.2f}'.format(curv_rad) + 'm'
        text2 = '{:04.2f}'.format(abs(center_dist)) + 'm ' + direction + ' of Lane Center'
        
        cv2.putText(result, text1, (40,55), font, 1, (200,255,155), 2, cv2.LINE_AA)
        cv2.putText(result, text2, (40,105), font, 1, (200,255,155), 2, cv2.LINE_AA)

        # -- 6.6. Result playback. -- # 
        cv2.imshow('Result', result)
#         cv2.imshow('Result', binimg) # Test bin_thresh for challenge videos
        out.write(result)

        # -- 6.7. Press q to exit. -- #
        if(cv2.waitKey(40) & 0xFF == ord('q')):
            break
            
    # --- 7. Free buffer. --- # 
    cap.release()
    out.release()
    cv2.destroyAllWindows()
        
    return 'ALF DONE!'

# ------------------------------------------------- MAIN ------------------------------------------------- # 

In [12]:
P2_Main('project_video.mp4', 'ALF_Output.mp4')
# P2_Main('challenge_video.mp4', 'ALF_Output_C.mp4')
# P2_Main('harder_challenge_video.mp4', 'ALF_Output_HC.mp4')

# Test
# P2_Main('project_video.mp4', 'Test.mp4')
# P2_Main('challenge_video.mp4', 'Test_C.mp4')
# P2_Main('harder_challenge_video.mp4', 'Test_HC.mp4')

ALF STARTED!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed, using the latest data!
Sanity check failed

'ALF DONE!'